In [1]:
from gurobipy import Model, GRB, tuplelist, quicksum
from gurobipy import read as gurobi_read_model
from multi_run_scheduler import *
import json
import random
import math
import os
import copy

In [2]:
s = MultiRunScheduler("sample_input/sample_input_v2_0.json")
s.progress_to_next_event()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-06
Set parameter MIPGap to value 0.01
Set parameter Method to value 3
Model constructed
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 793 rows, 1184 columns and 13424 nonzeros
Model fingerprint: 0x9378bb14
Variable types: 0 continuous, 1184 integer (1184 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7457.6142104
Presolve removed 793 rows and 1184 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 1:

In [3]:
class PossibleStart(object):
    def __init__(self, resource, slice_starts, internal_start, slice_size):
        self.resource = resource
        self.first_slice_start = slice_starts[0]
        self.all_slice_starts = slice_starts
        self.internal_start = internal_start
        self.slice_size = slice_size

    def __lt__(self, other):
        return self.first_slice_start < other.first_slice_start

    def __eq__(self, other):
        return self.first_slice_start == self.first_slice_start

    def __gt__(self, other):
        return self.first_slice_start > self.first_slice_start

    def __str__(self):
        return "resource_{}_start_{}_length_{}".format(self.resource,
                                                       self.first_slice_start,
                                                       self.slice_size)

In [110]:
def overlap_time_segments(seg1, seg2):
    overlap_segments = []
    i = 0
    j = 0

    while ((i < len(seg1)) and (j < len(seg2))):
        s1 = seg1[i]
        s2 = seg2[j]

        if s1["end"] < s2["start"]: # Current s1 is behind Current s2, move to next s1
            i += 1
            continue

        if s2["end"] < s1["start"]: # Current s2 is behind Current s1, move to next s2
            j += 1
            continue

        window_start = max([s1["start"], s2["start"]])
        
        if s1["end"] < s2["end"]: # s1 ends first, move to next s1
            window_end = s1["end"]
            i += 1
            
        elif s1["end"] > s2["end"]: # s2 ends first, move to next s2
            window_end= s2["end"]
            j += 1

        else: # Both segments end at the same time, move to next of both
            window_end = s1["end"]
            i += 1
            j+= 1

        if window_start != window_end:
            overlap_segments.append({"start": window_start, "end": window_end})
            
    return overlap_segments

In [111]:
class Injection(object):
    def __init__(self, injection_time, injection_type, data=None):
        self.injection_time = injection_time
        self.injection_type = injection_type
        self.data = data

    def __str__(self):
        return f"{self.injection_type}_{self.injection_time}"
    
    def __lt__(self, other):
        if self.injection_time == other.injection_time:
            return self.type_lt(other)
        else:
            return self.injection_time < other.injection_time

    def __gt__(self, other):
        if self.injection_time == other.injection_time:
            return self.type_gt(other)
        else:
            return self.injection_time > other.injection_time

    def __eq__(self, other):
        if self.injection_time == other.injection_time:
            return self.type_eq(other)
        else:
            return False

    def type_lt(self, other):
        if self.injection_type == "request": # Either Request vs Request, or Request vs Resource
            return False
        elif other.injection_type == "resource": # Either Resource vs Resource, or Request vs Resource
            return False
        else:
            return True # Must be Resource vs Request

    def type_gt(self, other):
        if self.injection_type == "resource":
            return False
        elif other.injection_type == "request":
            return False
        else: return True

    def type_eq(self, other):
        return self.injection_type == other.injection_type


    def to_json(self):
        return {
            "injection_time": self.injection_time,
            "injection_type": self.injection_type,
            "injection_data": self.data
        }


def injection_from_json(injection_json):
    return Injection(injection_json["injection_time"],
                     injection_json["injection_type"],
                     injection_json["injection_data"])


In [112]:
class TelescopeEvent(Injection):
    def __init__(self, injection_time, resource, closed):
        data = {"resource": resource, "closed": closed}
        super().__init__(injection_time, "resource", data)


class RequestEvent(Injection):
    def __init__(self, injection_time, data):
        super().__init__(injection_time, "request", data)

In [3]:
class Scheduler2(object):
    def __init__(self, input_filepath):
        self.load_file(input_filepath)

    def load_file(self, input_filepath):
        # Load initial parameters
        # Load telescope closure injections
        # Load Request injections
        # Fast forward to the first request injection?

        with open(input_filepath, "r") as f:
            input_data = json.load(f)

        # Load Initial Parameters
        self.start_time = input_data["input_parameters"]["start_time"]
        self.start_time = input_data["input_parameters"]["start_time"]
        self.horizon = input_data["input_parameters"]["horizon"]
        self.slice_size = input_data["input_parameters"]["slice_size"]
        self.proposals = input_data["input_parameters"]["proposals"]

        # Events that occur that the scheduler needs to respond to
        self.events = []

        # All the requests that the scheduler would have received if it had perfect information
        self.all_requests = {}
        self.all_closures = {}

        # The known, predictable state of resource availability
        self.base_resources = input_data["input_parameters"]["resources"]

        self.load_events(input_data["injections"])

        self.current_requests = []
        self.current_resources = [res for res in self.base_resources]
        
        self.next_event = 0
        self.last_event = len(self.events) - 1


    def load_events(self, injection_list):
        for inj in injection_list:
            if inj["injection_type"] == "request":
                new_requests = []
                for reqID in inj["injection_data"]:
                    self.all_requests[reqID] = inj["injection_data"][reqID]
                    new_requests.append(reqID)
                self.events.append(RequestEvent(inj["injection_time"], new_requests))

            elif inj["injection_type"] == "resource":
                pass
                # self.events.append(TelescopeEvent(inj.injection_data["start_time"],
                #                              inj.injection_data["resource"],
                #                              True)
                #              )
                # self.events.append(TelescopeEvent(inj.injection_data["end_time"],
                #                              inj.injection_data["resource"],
                #                              False)
                #              )
        self.events.sort()

    
    def progress_to_next_event(self):
        current_event = self.events[self.next_event]
        if current_event.injection_type == "request":
            self.process_request_event(current_event)
        elif current_event.injection_type == "resource":
            self.process_resource_event(current_event)

        self.now = current_event.injection_time
        
        self.run_scheduler()
        
        if self.next_event < self.last_event:
            self.next_event += 1
            self.progress_to_next_event()

        else:
            print("Scheduler has finished running.")
            return


    def process_request_event(self, event):
        for new_request_id in event.data:
            self.current_requests.append(new_request_id)
            


    def process_resource_event(self, event):
        pass


    def run_scheduler(self):
        # Determine requests
        requests = {}
        print(self.current_requests)
        print(self.all_requests)
        for reqID in self.current_requests:
            requests[reqID] = self.all_requests[reqID]

        # Determine resources
        resources = {}
        for res in self.current_resources:
            resources[res] = self.base_resources[res]
        
        sched = Scheduler(self.now, self.horizon, self.slice_size, resources, self.proposals, requests)
        sched.calculate_free_windows()
        sched.build_data_structures()
        sched.build_model()
        sched.solve_model()
        sched.interpret_solution()

In [4]:
s = Scheduler2("sample_input/sample_input_v2_0.json")

In [118]:
s.progress_to_next_event()

['0', '1', '2', '3', '4']
{'0': {'windows': {'t1': [{'start': 220537, 'end': 239241}]}, 'duration': 7747, 'proposal': 'proposal_2', 'resID': 0}, '1': {'windows': {'t1': [{'start': 51264, 'end': 105106}, {'start': 124276, 'end': 125873}, {'start': 142813, 'end': 145082}, {'start': 188996, 'end': 220273}]}, 'duration': 10530, 'proposal': 'proposal_1', 'resID': 1}, '2': {'windows': {'t1': [{'start': 4970, 'end': 40746}, {'start': 103219, 'end': 138148}, {'start': 167424, 'end': 195314}, {'start': 228552, 'end': 243702}]}, 'duration': 1109, 'proposal': 'proposal_1', 'resID': 2}, '3': {'windows': {'t1': [{'start': 79975, 'end': 205492}]}, 'duration': 568, 'proposal': 'proposal_2', 'resID': 3}, '4': {'windows': {'t1': [{'start': 102609, 'end': 104537}, {'start': 112918, 'end': 117554}, {'start': 152233, 'end': 156910}, {'start': 188204, 'end': 189435}, {'start': 191873, 'end': 207598}, {'start': 210953, 'end': 231375}, {'start': 242213, 'end': 242460}, {'start': 246258, 'end': 253731}]}, 'du